In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("C:/Users/sumit/Downloads/titanic.csv")
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [10]:
#How many siblings/spouses aboard (SibSp) had survivors? What’s the survival rate for each SibSp count?

SibSp_survived = data.groupby("SibSp")["Survived"].agg(["count","sum"])
SibSp_survived_rate = (SibSp_survived["sum"]/SibSp_survived["count"])*100
SibSp_survived_rate

SibSp
0    34.539474
1    53.588517
2    46.428571
3    25.000000
4    16.666667
5     0.000000
8     0.000000
dtype: float64

In [20]:
#Which ticket prefixes are most common? (Some tickets have prefixes like 'PC', 'STON', etc.)
data["ticket_prefixes"] = data["Ticket"].str.extract(r'(^[A-Za-z\.\/]+)')
ticket_prefixes_common = data["ticket_prefixes"].value_counts()
ticket_prefixes_common.head(5)

ticket_prefixes
PC        60
C.A.      27
A/        23
STON/O    18
W./C.      9
Name: count, dtype: int64

In [28]:
#Create a new column FamilySize = SibSp + Parch + 1. How does survival vary by family size?
data["FamilySize"] = data["SibSp"] + data["Parch"] + 1
survival_by_family = data.groupby("FamilySize")["Survived"].agg(["count" , "sum"])
survival_by_family_rate = (survival_by_family["sum"]/survival_by_family["count"])*100
survival_by_family_rate

FamilySize
1     30.353818
2     55.279503
3     57.843137
4     72.413793
5     20.000000
6     13.636364
7     33.333333
8      0.000000
11     0.000000
dtype: float64

In [31]:
#Create a new column: IsAlone (1 if FamilySize == 1, else 0). What is the survival rate for people traveling alone vs. not?
data["IsAlone"] = (data["FamilySize"]==1).astype(int)
survival_rate = data.groupby("IsAlone")["Survived"].agg(["count","sum"])
survival_rate_alone = (survival_rate["sum"]/survival_rate["count"])*100        #(sum/count)*100	Converts the survival rate to percent
survival_rate_alone

#Use agg(['count', 'sum']) when you want to:
#Analyze survival rate
#Keep group-wise structure
#Perform further calculations
#Use value_counts() only when you want detailed breakdowns, but not for rate calculations directly.

IsAlone
0    50.564972
1    30.353818
dtype: float64

In [37]:
#Bin the Fare column into 4 quantiles using pd.qcut(). Check survival rates per fare bin.

data["Fare_column"] = pd.qcut(data["Fare"],q = 4)              #pd.qcut():You want to divide data by distribution, like quartiles or deciles.
Fare_column_agg = data.groupby("Fare_column")["Survived"].agg(["count","sum"])
Fare_column_agg_rate = (Fare_column_agg["sum"]/Fare_column_agg["count"])*100
Fare_column_agg_rate

C:\Users\sumit\AppData\Local\Temp\ipykernel_15976\3479872358.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  Fare_column_agg = data.groupby("Fare_column")["Survived"].agg(["count","sum"])


Fare_column
(-0.001, 7.91]     19.730942
(7.91, 14.454]     30.357143
(14.454, 31.0]     45.495495
(31.0, 512.329]    58.108108
dtype: float64

In [41]:
#What was the survival rate for each cabin group: "Cabin Known" vs "Cabin Unknown"?
#Create a new column that flags whether a passenger had cabin info or not.
#Group by this flag and calculate survival rate.

data["Cabin_known"] = data["Cabin"].notnull().astype(int)
cabin_survival = data.groupby("Cabin_known")["Survived"].agg(["count","sum"])
cabin_survival_rate = (cabin_survival["sum"]/cabin_survival["count"])*100
cabin_survival_rate

Cabin_known
0    29.985444
1    66.666667
dtype: float64

In [44]:
#Which Embarked port had the highest survival rate?
Embarked_port = data.groupby("Embarked")["Survived"].agg(["count","sum"])
Embarked_port_rate = (Embarked_port["sum"]/Embarked_port["count"])*100
Embarked_port_rate

Embarked
C    55.357143
Q    38.961039
S    33.695652
dtype: float64